# `gmxapi` Practice
In this notebook, our goal is to use `gmxapi` to run a standard MD simulation, an expanded ensemble simulaiton, and a 1D alchemical metadynamics simulation. Note that before running this notebook, one should first load in GROMACS and activate corresponding virtual environment as needed. 

In [1]:
import sys
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

## Section 1. Running a standard MD simulation using `gmxapi`

To get started, we set up the directories as below:

In [2]:
%%bash
mkdir standard_MD && cd standard_MD
cp -r ../inputs/* .
mkdir Box Sol EM Equil MD Topology
mv *top Topology/.
cd EM && cp ../mdp_files/em.mdp .
cd ../Equil && mkdir NVT && cd NVT && cp ../../mdp_files/nvt_equil.mdp .
cd ../ && mkdir NPT && cd NPT && cp ../../mdp_files/npt_equil.mdp .
cd ../../MD && cp ../mdp_files/md.mdp .

### 1. Construct the simulation box
Here we use `sys2_init.gro` as the input file, which is the same as `simple_system2.gro` except that the box vector was removed. Normally, the GROMACS command would be as follows if the working directory is the path of this notebook:
```
gmx editconf -f standard_MD/sys2_init.gro -o standard_MD/Box/sys2_box.gro -bt dodecahedron -d 1 -c
```
Using `gmxapi`, we first create an object using `gmx.commandline_operation` and run `object.run()`:

In [3]:
box = gmx.commandline_operation('gmx',
                               arguments=['editconf', '-bt', 'dodecahedron', '-d', '1', '-c'],
                               input_files={'-f': '../standard_MD/sys2_init.gro'},
                               output_files={'-o': '../standard_MD/Box/sys2_box.gro'})
box.run()

Note that `box.run()` will create a folder `gmxapi.commandline.cli*_i0` and use it as the working directory, so the path to the input files and output files should be `../standard_MD/sys2_init.gro`. Additionally, if `output_files` is not specified, then the default file names will be used and the outputs will be samed in the newly created folder `gmxapi.commandline.cli*_i0`. Here, we specify the path of the output so that the folder `gmxapi.commandline.cli*_i0` will be empty after the command is finished.

To check if the command was executed successfully, we can check `returncode` as follows. Additionally, we can check `stdout` and `stderr` to get more information about the output.

In [4]:
print(f'Return code of the process: {box.output.returncode.result()}\n')
print(f'STDERR of the process:\n\n {box.output.stderr.result()}\n')
print(f'STDOUT of the process:\n\n {box.output.stdout.result()}\n')

Return code of the process: 0

STDERR of the process:

               :-) GROMACS - gmx editconf, 2021.4-plumed-2.7.3 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
      

We can also check the filename of the output and the flag where it was specified as follows:

In [5]:
print(box.output.file.result())

{'-o': '/home/wei-tse/Documents/gmxapi_practice/standard_MD/Box/sys2_box.gro'}


### 2. Solvate the system

Again, to solvate the system, the original GROMACS command is as follows:
```
gmx solvate -cp standard_MD/Box/sys2_box.gro -p standard_MD/Topology/sys2.top -o sys2_sol.gro -cs
```
Therefore, we can execute the command as follows. Note that now we could either explicitly specify the output file of the previous command or use the `box` object to retrieve the output filename in the last step. Also note that `box.output.file['-o']` is a `gmxapi.operation.Future` object, and `box.output.file['-o'].result()`.
```
>>> print(box.output.file['-o'])
<Future: name='file', description=ResultDescription(dtype=None, width=1)>
```

In [6]:
solvate = gmx.commandline_operation('gmx',
                                   arguments=['solvate', '-cs'],
                                   input_files={
                                       '-cp': box.output.file['-o'],
                                       '-p': '../standard_MD/Topology/sys2.top'},
                                   output_files={'-o': '../standard_MD/Sol/sys2_sol.gro'})
solvate.run()

In [7]:
if solvate.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {solvate.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')

The process was executed successfully.


### 3. Energy minimization

Again, we run the following GROMACS commands using `gmxapi`:
```
gmx grompp -f standard_MD/EM/em.mdp -c standard_MD/Sol/sys2_sol.gro -p standard_MD/Topology/sys2.top -o standard_MD/EM/sys2_em.tpr
gmx mdrun -s standard_MD/EM/sys2_em.tpr -c standard_MD/EM/sys2_em.gro -g standard_MD/EM/em.log -e standard_MD/EM/em.edr
```

In [8]:
grompp_em = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/EM/em.mdp',
                                         '-c': '../standard_MD/Sol/sys2_sol.gro',
                                         '-p': '../standard_MD/Topology/sys2.top'},
                                     output_files={'-o': '../standard_MD/EM/sys2_em.tpr'})
grompp_em.run()
if grompp_em.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {grompp_em.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')

The process was executed successfully.


There are two ways to launch an `mdrun` command. The first way is to use `commandline_operation` as before. This time we demonstrate how one could use `grompp.output.file['-o']` as an input here. 

In [9]:
em = gmx.commandline_operation('gmx',
                              arguments=['mdrun'],
                              input_files={'-s': grompp_em.output.file['-o']},
                              output_files={
                                  '-c': '../standard_MD/EM/sys2_em.gro',
                                  '-e': '../standard_MD/EM/em.edr',
                                  '-g': '../standard_MD/EM/em.log',
                                  '-o': '../standard_MD/EM/em.trr'})
em.run()
if em.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {em.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')

The process was executed successfully.


Another way to run a `mdrun` command using `gmxapi` is to use `gmx.mdrun`:
```
em_input = gmx.read_tpr(grompp_em.output.file['-o'])
# em_modified = gmx.modify_input(input=em_input, parameters={'nsteps': 100})  # modify the parameters as needed, only work with gmx.mdrun (not gmx.commandline_operation)
em = gmx.mdrun(input=em_input)
em.run()
```
Or simply
```
em = gmx.mdrun(input=grompp_em.output.file['-o'])
em.run()
```
Note that the argument `input` should be the tpr file and `gmx.mdrun` creates a folder named as `mdrun_*_i0_0/`.It doesn't seem that we can specify the output file names though ...

### 4. Equilibration

To run NVT and then NPT equilibration using GROMACS, we use the following commands:
```
gmx grompp -f standard_MD/Equil/NVT/nvt_equil.mdp -c standard_MD/EM/sys2_em.gro -p standard_MD/Topology/sys2.top -o standard_MD/Equil/NVT/sys2_equil.tpr
gmx mdrun -s standard_MD/Equil/NVT/sys2_equil.tpr -c standard_MD/Equil/NVT/sys2_equil.gro -g standard_MD/Equil/NVT/equil.log -e standard_MD/Equil/NVT/equil.edr
gmx grompp -f npt_equil.mdp -c ../NVT/sys2_equil.gro -t ../NVT/state.cpt -p ../../Topology/sys2.top -o sys2_equil.tpr
gmx mdrun -s sys2_equil.tpr -c sys2_equil.gro -g equil.log -e equil.edr
```
Below we first define a simple function to deal with `stderr` first, and then we execute the commands above using `gmxapi`. Note that the equilibration length was very short because this notebook is just for demonstration purpose. Also note that the will be an `xtc` file output by `mdrun` but it will be saved in the newly-created folder since we don't specify it in `output_files`.

In [10]:
def gmx_output(obj):
    if obj.output.returncode.result() != 0:
        print(f'STDERR of the process:\n\n {obj.output.stderr.result()}\n')
    else:
        print('The process was executed successfully.')

In [11]:
print('Performing NVT equilibration ...')
grompp_nvt = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/Equil/NVT/nvt_equil.mdp',
                                         '-c': '../standard_MD/EM/sys2_em.gro',
                                         '-p': '../standard_MD/Topology/sys2.top'},
                                     output_files={'-o': '../standard_MD/Equil/NVT/sys2_equil.tpr'})
grompp_nvt.run()
gmx_output(grompp_nvt)
    
nvt = gmx.commandline_operation('gmx',
                               arguments=['mdrun'],
                               input_files={'-s': grompp_nvt.output.file['-o']},
                               output_files={
                                   '-c': '../standard_MD/Equil/NVT/sys2_equil.gro',
                                   '-e': '../standard_MD/Equil/NVT/equil.edr',
                                   '-g': '../standard_MD/Equil/NVT/equil.log',
                                   '-o': '../standard_MD/Equil/NVT/equil.trr',
                                   '-cpo': '../standard_MD/Equil/NVT/state.cpt'})
nvt.run()
gmx_output(nvt)
    
print('\nPerforming NPT equilibration ...')
grompp_npt = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/Equil/NPT/npt_equil.mdp',
                                         '-c': '../standard_MD/Equil/NVT/sys2_equil.gro',
                                         '-p': '../standard_MD/Topology/sys2.top',
                                         '-t': '../standard_MD/Equil/NVT/state.cpt'},
                                     output_files={'-o': '../standard_MD/Equil/NPT/sys2_equil.tpr'})
grompp_npt.run()
gmx_output(grompp_npt)
    
npt = gmx.commandline_operation('gmx',
                               arguments=['mdrun'],
                               input_files={'-s': grompp_npt.output.file['-o']},
                               output_files={
                                   '-c': '../standard_MD/Equil/NPT/sys2_equil.gro',
                                   '-e': '../standard_MD/Equil/NPT/equil.edr',
                                   '-g': '../standard_MD/Equil/NPT/equil.log',
                                   '-o': '../standard_MD/Equil/NPT/equil.trr',
                                   '-cpo': '../standard_MD/Equil/NPT/state.cpt'})
npt.run()
gmx_output(npt)

Performing NVT equilibration ...
The process was executed successfully.
The process was executed successfully.

Performing NPT equilibration ...
The process was executed successfully.
The process was executed successfully.


### 5. Launch the simulation

Lastly, we use the `gmxapi` to run the following commands:
```
gmx grompp -f standard_MD/MD/md.mdp -c standard_MD/Equil/NPT/sys2_equil.gro -p ../Topology/sys2.top -o sys2_md.tpr -t ../Equil/NPT/state.cpt
gmx mdrun -s sys2_md.tpr -x sys2_md.xtc -c sys2_md.gro -g md.log -e md.edr
```

In [12]:
grompp_md = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/MD/md.mdp',
                                         '-c': '../standard_MD/Equil/NPT/sys2_equil.gro',
                                         '-p': '../standard_MD/Topology/sys2.top',
                                         '-t': '../standard_MD/Equil/NPT/state.cpt'},
                                     output_files={'-o': '../standard_MD/MD/sys2_md.tpr'})
grompp_md.run()
gmx_output(grompp_md)
    
md = gmx.commandline_operation('gmx',
                              arguments=['mdrun'],
                              input_files={'-s': grompp_npt.output.file['-o']},
                              output_files={
                                  '-c': '../standard_MD/MD/sys2_md.gro',
                                  '-e': '../standard_MD/MD/md.edr',
                                  '-g': '../standard_MD/MD/md.log',
                                  '-o': '../standard_MD/MD/md.trr'})
npt.run()
gmx_output(npt)

The process was executed successfully.
The process was executed successfully.
